# 0. GAN
- reference : https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/gan/gan.py
- paper : https://arxiv.org/abs/1406.2661

# 1. Library Import

In [27]:
# import argparse -> jupyter notebook X
import easydict

import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [11]:
# make images folder
os.makedirs('images', exist_ok = True)

# 2. Parameter Setting

In [28]:
opt = easydict.EasyDict({"n_epochs" : 200,
                         "batch_size" : 64,
                         "lr" : 0.0002,
                         "b1" : 0.5,
                         "b2" : 0.99,
                         "n_cpu" : 8,
                         "latent_dim" : 100,
                         "img_size" : 28,
                         "channels" : 1,
                         "sample_interval" : 400})

In [29]:
print(opt)

{'n_epochs': 200, 'batch_size': 64, 'lr': 0.0002, 'b1': 0.5, 'b2': 0.99, 'n_cpu': 8, 'latent_dim': 100, 'img_size': 28, 'channels': 1, 'sample_interval': 400}


In [30]:
img_shape = (opt.channels, opt.img_size, opt.img_size)

In [32]:
print(img_shape)

(1, 28, 28)


In [33]:
cuda = True if torch.cuda.is_available() else False

In [34]:
print(cuda)

False


# 3. Model

### 3.1 Generator

In [35]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        def block(in_feat, out_feat, normalize = True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace = True))
            return layers
        
        self.model = nn.Sequential(*block(opt.latent_dim, 128, normalize = False),
                                   *block(128, 256),
                                   *block(256, 512),
                                   *block(512, 1024),
                                   nn.Linear(1024, int(np.prod(img_shape))),
                                   nn.Tanh()
                                  )
        
    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

### 3.2 Discriminator

In [36]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.model = nn.Sequential(nn.Linear(int(np.prod(img_shape)), 512),
                                   nn.LeakyReLU(0.2, inplace = True),
                                   nn.Linear(512, 256),
                                   nn.LeakyReLU(0.2, inplace = True),
                                   nn.Linear(256, 1),
                                   nn.Sigmoid()
                                  )
        
    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

### 3.3 Loss Function

In [37]:
adversarial_loss = torch.nn.BCELoss()

# 4. Data Loader and Model Training

In [38]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [39]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

### 4.1 Data Loader (MNIST)

In [43]:
# data loader
os.makedirs("../data/mnist", exist_ok = True)
dataloader = torch.utils.data.DataLoader(
            datasets.MNIST(
                "../data/mnist",
                train = True,
                download = True,
                transform = transforms.Compose(
                    [transforms.Resize(opt.img_size), transforms.ToTensor(),
                     transforms.Normalize([0.5], [0.5])]
                )
            ),
            batch_size = opt.batch_size,
            shuffle = True
            )

### 4.2 Model Training

In [46]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr = opt.lr, betas = (opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr = opt.lr, betas = (opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad = False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad = False)
        
        # Confiqure input
        real_imgs = Variable(imgs.type(Tensor))
        
        ###################
        # Train Generator #
        ###################
        optimizer_G.zero_grad()
        
        # sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))
        
        # Generate a batch of images
        gen_imgs = generator(z)
        
        # Loss measures generator's ability to fool discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        
        g_loss.backward()
        optimizer_G.step()
        
        #######################
        # Train Discriminator #
        #######################
        optimizer_D.zero_grad()
        
        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        
        d_loss.backward()
        optimizer_D.step()
        
        print("[Epoch %d / %d] [Batch %d / %d] [D loss: %f] [G loss: %f]"
              % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
             )
        
        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow = 5, normalize = True)

[Epoch 0 / 200] [Batch 0 / 938] [D loss: 0.645640] [G loss: 0.715141]
[Epoch 0 / 200] [Batch 1 / 938] [D loss: 0.572171] [G loss: 0.712260]
[Epoch 0 / 200] [Batch 2 / 938] [D loss: 0.513467] [G loss: 0.709461]
[Epoch 0 / 200] [Batch 3 / 938] [D loss: 0.464812] [G loss: 0.706187]
[Epoch 0 / 200] [Batch 4 / 938] [D loss: 0.427159] [G loss: 0.701891]
[Epoch 0 / 200] [Batch 5 / 938] [D loss: 0.406876] [G loss: 0.697024]
[Epoch 0 / 200] [Batch 6 / 938] [D loss: 0.394099] [G loss: 0.690256]
[Epoch 0 / 200] [Batch 7 / 938] [D loss: 0.382485] [G loss: 0.680869]
[Epoch 0 / 200] [Batch 8 / 938] [D loss: 0.380297] [G loss: 0.671190]
[Epoch 0 / 200] [Batch 9 / 938] [D loss: 0.379243] [G loss: 0.660861]
[Epoch 0 / 200] [Batch 10 / 938] [D loss: 0.385736] [G loss: 0.646703]
[Epoch 0 / 200] [Batch 11 / 938] [D loss: 0.387154] [G loss: 0.639220]
[Epoch 0 / 200] [Batch 12 / 938] [D loss: 0.395768] [G loss: 0.622326]
[Epoch 0 / 200] [Batch 13 / 938] [D loss: 0.401470] [G loss: 0.613614]
[Epoch 0 / 200] 

[Epoch 0 / 200] [Batch 118 / 938] [D loss: 0.401579] [G loss: 1.032117]
[Epoch 0 / 200] [Batch 119 / 938] [D loss: 0.418226] [G loss: 1.123207]
[Epoch 0 / 200] [Batch 120 / 938] [D loss: 0.444070] [G loss: 0.907819]
[Epoch 0 / 200] [Batch 121 / 938] [D loss: 0.429459] [G loss: 1.061284]
[Epoch 0 / 200] [Batch 122 / 938] [D loss: 0.405871] [G loss: 1.086295]
[Epoch 0 / 200] [Batch 123 / 938] [D loss: 0.397822] [G loss: 1.270712]
[Epoch 0 / 200] [Batch 124 / 938] [D loss: 0.423207] [G loss: 1.008859]
[Epoch 0 / 200] [Batch 125 / 938] [D loss: 0.413329] [G loss: 1.176769]
[Epoch 0 / 200] [Batch 126 / 938] [D loss: 0.462466] [G loss: 0.993475]
[Epoch 0 / 200] [Batch 127 / 938] [D loss: 0.442832] [G loss: 0.901620]
[Epoch 0 / 200] [Batch 128 / 938] [D loss: 0.496767] [G loss: 1.169475]
[Epoch 0 / 200] [Batch 129 / 938] [D loss: 0.531696] [G loss: 0.681208]
[Epoch 0 / 200] [Batch 130 / 938] [D loss: 0.535556] [G loss: 1.131392]
[Epoch 0 / 200] [Batch 131 / 938] [D loss: 0.521260] [G loss: 0.

[Epoch 0 / 200] [Batch 234 / 938] [D loss: 0.546800] [G loss: 0.621239]
[Epoch 0 / 200] [Batch 235 / 938] [D loss: 0.508718] [G loss: 0.877225]
[Epoch 0 / 200] [Batch 236 / 938] [D loss: 0.491633] [G loss: 0.857189]
[Epoch 0 / 200] [Batch 237 / 938] [D loss: 0.478287] [G loss: 0.824910]
[Epoch 0 / 200] [Batch 238 / 938] [D loss: 0.462787] [G loss: 0.980766]
[Epoch 0 / 200] [Batch 239 / 938] [D loss: 0.459399] [G loss: 0.936609]
[Epoch 0 / 200] [Batch 240 / 938] [D loss: 0.472321] [G loss: 0.884033]
[Epoch 0 / 200] [Batch 241 / 938] [D loss: 0.459597] [G loss: 0.920591]
[Epoch 0 / 200] [Batch 242 / 938] [D loss: 0.477393] [G loss: 0.908603]
[Epoch 0 / 200] [Batch 243 / 938] [D loss: 0.454361] [G loss: 0.859875]
[Epoch 0 / 200] [Batch 244 / 938] [D loss: 0.500454] [G loss: 0.950302]
[Epoch 0 / 200] [Batch 245 / 938] [D loss: 0.525852] [G loss: 0.691926]
[Epoch 0 / 200] [Batch 246 / 938] [D loss: 0.527471] [G loss: 1.033368]
[Epoch 0 / 200] [Batch 247 / 938] [D loss: 0.571554] [G loss: 0.

[Epoch 0 / 200] [Batch 348 / 938] [D loss: 0.568636] [G loss: 1.277592]
[Epoch 0 / 200] [Batch 349 / 938] [D loss: 0.517664] [G loss: 0.622351]
[Epoch 0 / 200] [Batch 350 / 938] [D loss: 0.486800] [G loss: 0.934696]
[Epoch 0 / 200] [Batch 351 / 938] [D loss: 0.471580] [G loss: 0.974955]
[Epoch 0 / 200] [Batch 352 / 938] [D loss: 0.522554] [G loss: 0.699082]
[Epoch 0 / 200] [Batch 353 / 938] [D loss: 0.463778] [G loss: 1.036011]
[Epoch 0 / 200] [Batch 354 / 938] [D loss: 0.477133] [G loss: 0.919322]
[Epoch 0 / 200] [Batch 355 / 938] [D loss: 0.495683] [G loss: 0.828765]
[Epoch 0 / 200] [Batch 356 / 938] [D loss: 0.494803] [G loss: 0.875017]
[Epoch 0 / 200] [Batch 357 / 938] [D loss: 0.526914] [G loss: 0.885712]
[Epoch 0 / 200] [Batch 358 / 938] [D loss: 0.549756] [G loss: 0.699607]
[Epoch 0 / 200] [Batch 359 / 938] [D loss: 0.582909] [G loss: 0.944780]
[Epoch 0 / 200] [Batch 360 / 938] [D loss: 0.602730] [G loss: 0.516779]
[Epoch 0 / 200] [Batch 361 / 938] [D loss: 0.618724] [G loss: 1.

[Epoch 0 / 200] [Batch 463 / 938] [D loss: 0.491584] [G loss: 1.009501]
[Epoch 0 / 200] [Batch 464 / 938] [D loss: 0.516696] [G loss: 0.793671]
[Epoch 0 / 200] [Batch 465 / 938] [D loss: 0.500283] [G loss: 1.027988]
[Epoch 0 / 200] [Batch 466 / 938] [D loss: 0.523698] [G loss: 0.694629]
[Epoch 0 / 200] [Batch 467 / 938] [D loss: 0.519153] [G loss: 1.277651]
[Epoch 0 / 200] [Batch 468 / 938] [D loss: 0.637798] [G loss: 0.426983]
[Epoch 0 / 200] [Batch 469 / 938] [D loss: 0.621960] [G loss: 1.792172]
[Epoch 0 / 200] [Batch 470 / 938] [D loss: 0.701641] [G loss: 0.338024]
[Epoch 0 / 200] [Batch 471 / 938] [D loss: 0.498307] [G loss: 1.389047]
[Epoch 0 / 200] [Batch 472 / 938] [D loss: 0.477643] [G loss: 0.786135]
[Epoch 0 / 200] [Batch 473 / 938] [D loss: 0.463214] [G loss: 0.936912]
[Epoch 0 / 200] [Batch 474 / 938] [D loss: 0.447417] [G loss: 1.009673]
[Epoch 0 / 200] [Batch 475 / 938] [D loss: 0.482931] [G loss: 0.931118]
[Epoch 0 / 200] [Batch 476 / 938] [D loss: 0.477499] [G loss: 0.

...

[Epoch 10 / 200] [Batch 296 / 938] [D loss: 0.329901] [G loss: 0.905356]
[Epoch 10 / 200] [Batch 297 / 938] [D loss: 0.395336] [G loss: 4.287791]
[Epoch 10 / 200] [Batch 298 / 938] [D loss: 0.183794] [G loss: 2.209330]
[Epoch 10 / 200] [Batch 299 / 938] [D loss: 0.273232] [G loss: 0.959540]
[Epoch 10 / 200] [Batch 300 / 938] [D loss: 0.200715] [G loss: 4.244139]
[Epoch 10 / 200] [Batch 301 / 938] [D loss: 0.165563] [G loss: 3.312045]
[Epoch 10 / 200] [Batch 302 / 938] [D loss: 0.266993] [G loss: 1.021525]
[Epoch 10 / 200] [Batch 303 / 938] [D loss: 0.273393] [G loss: 3.931867]
[Epoch 10 / 200] [Batch 304 / 938] [D loss: 0.115528] [G loss: 2.464188]
[Epoch 10 / 200] [Batch 305 / 938] [D loss: 0.166167] [G loss: 1.639620]
[Epoch 10 / 200] [Batch 306 / 938] [D loss: 0.135959] [G loss: 2.523922]
[Epoch 10 / 200] [Batch 307 / 938] [D loss: 0.147431] [G loss: 2.293468]
[Epoch 10 / 200] [Batch 308 / 938] [D loss: 0.174691] [G loss: 1.742679]
[Epoch 10 / 200] [Batch 309 / 938] [D loss: 0.25081

[Epoch 10 / 200] [Batch 411 / 938] [D loss: 0.278215] [G loss: 1.358031]
[Epoch 10 / 200] [Batch 412 / 938] [D loss: 0.232939] [G loss: 2.196145]
[Epoch 10 / 200] [Batch 413 / 938] [D loss: 0.298512] [G loss: 1.488814]
[Epoch 10 / 200] [Batch 414 / 938] [D loss: 0.206498] [G loss: 2.023359]
[Epoch 10 / 200] [Batch 415 / 938] [D loss: 0.244017] [G loss: 1.946427]
[Epoch 10 / 200] [Batch 416 / 938] [D loss: 0.268252] [G loss: 1.393839]
[Epoch 10 / 200] [Batch 417 / 938] [D loss: 0.173712] [G loss: 2.713122]
[Epoch 10 / 200] [Batch 418 / 938] [D loss: 0.307747] [G loss: 2.236683]
[Epoch 10 / 200] [Batch 419 / 938] [D loss: 0.544997] [G loss: 0.574081]
[Epoch 10 / 200] [Batch 420 / 938] [D loss: 0.781805] [G loss: 6.202078]
[Epoch 10 / 200] [Batch 421 / 938] [D loss: 0.160302] [G loss: 2.126749]
[Epoch 10 / 200] [Batch 422 / 938] [D loss: 0.431200] [G loss: 0.793149]
[Epoch 10 / 200] [Batch 423 / 938] [D loss: 0.244057] [G loss: 4.543092]
[Epoch 10 / 200] [Batch 424 / 938] [D loss: 0.19905

[Epoch 10 / 200] [Batch 526 / 938] [D loss: 0.481453] [G loss: 0.664921]
[Epoch 10 / 200] [Batch 527 / 938] [D loss: 0.552128] [G loss: 5.333972]
[Epoch 10 / 200] [Batch 528 / 938] [D loss: 0.225971] [G loss: 2.139922]
[Epoch 10 / 200] [Batch 529 / 938] [D loss: 0.398490] [G loss: 0.935398]
[Epoch 10 / 200] [Batch 530 / 938] [D loss: 0.362970] [G loss: 4.452122]
[Epoch 10 / 200] [Batch 531 / 938] [D loss: 0.232501] [G loss: 2.076166]
[Epoch 10 / 200] [Batch 532 / 938] [D loss: 0.461254] [G loss: 0.671222]
[Epoch 10 / 200] [Batch 533 / 938] [D loss: 0.481231] [G loss: 4.935853]
[Epoch 10 / 200] [Batch 534 / 938] [D loss: 0.184661] [G loss: 2.697130]
[Epoch 10 / 200] [Batch 535 / 938] [D loss: 0.405275] [G loss: 0.714914]
[Epoch 10 / 200] [Batch 536 / 938] [D loss: 0.344302] [G loss: 3.890998]
[Epoch 10 / 200] [Batch 537 / 938] [D loss: 0.304532] [G loss: 2.158020]
[Epoch 10 / 200] [Batch 538 / 938] [D loss: 0.481814] [G loss: 0.772507]
[Epoch 10 / 200] [Batch 539 / 938] [D loss: 0.63489

[Epoch 10 / 200] [Batch 638 / 938] [D loss: 0.318064] [G loss: 1.680054]
[Epoch 10 / 200] [Batch 639 / 938] [D loss: 0.316913] [G loss: 1.134180]
[Epoch 10 / 200] [Batch 640 / 938] [D loss: 0.142035] [G loss: 3.100835]
[Epoch 10 / 200] [Batch 641 / 938] [D loss: 0.216437] [G loss: 2.627998]
[Epoch 10 / 200] [Batch 642 / 938] [D loss: 0.354480] [G loss: 1.185812]
[Epoch 10 / 200] [Batch 643 / 938] [D loss: 0.302484] [G loss: 2.958277]
[Epoch 10 / 200] [Batch 644 / 938] [D loss: 0.255602] [G loss: 1.306025]
[Epoch 10 / 200] [Batch 645 / 938] [D loss: 0.373082] [G loss: 2.187612]
[Epoch 10 / 200] [Batch 646 / 938] [D loss: 0.242648] [G loss: 1.259111]
[Epoch 10 / 200] [Batch 647 / 938] [D loss: 0.182508] [G loss: 2.735482]
[Epoch 10 / 200] [Batch 648 / 938] [D loss: 0.234078] [G loss: 1.921050]
[Epoch 10 / 200] [Batch 649 / 938] [D loss: 0.269377] [G loss: 1.543701]
[Epoch 10 / 200] [Batch 650 / 938] [D loss: 0.238485] [G loss: 2.134777]
[Epoch 10 / 200] [Batch 651 / 938] [D loss: 0.35139

[Epoch 10 / 200] [Batch 750 / 938] [D loss: 0.277956] [G loss: 2.346191]
[Epoch 10 / 200] [Batch 751 / 938] [D loss: 0.338310] [G loss: 0.919431]
[Epoch 10 / 200] [Batch 752 / 938] [D loss: 0.411758] [G loss: 3.648042]
[Epoch 10 / 200] [Batch 753 / 938] [D loss: 0.255121] [G loss: 1.231332]
[Epoch 10 / 200] [Batch 754 / 938] [D loss: 0.217205] [G loss: 2.300177]
[Epoch 10 / 200] [Batch 755 / 938] [D loss: 0.198285] [G loss: 1.682531]
[Epoch 10 / 200] [Batch 756 / 938] [D loss: 0.255141] [G loss: 2.348511]
[Epoch 10 / 200] [Batch 757 / 938] [D loss: 0.264860] [G loss: 1.420367]
[Epoch 10 / 200] [Batch 758 / 938] [D loss: 0.231194] [G loss: 2.071715]
[Epoch 10 / 200] [Batch 759 / 938] [D loss: 0.251908] [G loss: 1.716647]
[Epoch 10 / 200] [Batch 760 / 938] [D loss: 0.280767] [G loss: 1.709457]
[Epoch 10 / 200] [Batch 761 / 938] [D loss: 0.241800] [G loss: 1.690670]
[Epoch 10 / 200] [Batch 762 / 938] [D loss: 0.246322] [G loss: 2.065881]
[Epoch 10 / 200] [Batch 763 / 938] [D loss: 0.26524